### Questions

### Objectives
YWBAT
* scrape data from a website (pagination)
* grab the useful data from a webpage
* store this data in a dataframe
* save your dataframe

### Outline

In [6]:
import requests # grabbing the 'html-info'  from the url

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup # a tool to traverse your html page or xml page or ... 


import matplotlib.pyplot as plt

In [7]:
search = 'floor lamps'
url = "https://www.ebay.com/sch/i.html?_nkw={}".format(search.replace(" ", "+"))
url
page_url = "https://www.ebay.com/sch/i.html?_nkw={}&_pgn={}".format(search.replace(" ", "+"), 2)

'https://www.ebay.com/sch/i.html?_nkw=floor+lamps'

In [8]:
res_page = requests.get(url)
res_page.content[:50]

b'<!DOCTYPE html><!--[if IE 9]><html class="ie9" lan'

In [9]:
soup = BeautifulSoup(res_page.content, 'html.parser')

In [11]:
# Let's build a function to make a soup object from a search

In [61]:
def get_ebay_soup(search='floor lamps', page_number=1, parser='html.parser'):
    url = "https://www.ebay.com/sch/i.html?_nkw={}&_pgn={}".format(search.replace(" ", "+"), page_number)
    res_page = requests.get(url)
    soup = BeautifulSoup(res_page.content, 'html.parser')
    return soup

In [62]:
def get_item_boxes(search, page, parser='html.parser'):
    soup = get_ebay_soup(search, page, parser)
    item_boxes = soup.find_all("div", attrs={"class":"s-item__wrapper"})
    return item_boxes

In [72]:
def make_item_box_dict_list(item_boxes, dlist=[]):
    for index, item_box in enumerate(item_boxes):
        d = dict()

        try:
            d["price"] = float(item_box.find("span", class_="s-item__price").text.strip("$"))
        except:
            # maybe include average price
            continue

        d["condition"] = item_box.find("span", class_="SECONDARY_INFO").text

        name_text = item_box.find("h3").text.replace("SPONSORED", "SPONSORED$&~")
        if 'SPONSORED' in name_text:
            d["sponsored"] = name_text.replace("SPONSORED", "SPONSORED ").split("$&~")[0]
            d["name"] = name_text.replace("SPONSORED", "SPONSORED ").split("$&~")[1]
        else:
            d["name"] = name_text
            d["sponsored"] = None

        # shipping price
        d["shipping_price"] = item_box.find("span", class_="s-item__shipping s-item__logisticsCost").text

        dlist.append(d)
    return dlist

In [73]:
def get_item_dict_list(search='floor lamps', page=1, parser='html.parser'):
    item_boxes = get_item_boxes(search=search, page=page, parser=parser)
    item_dict_list = make_item_box_dict_list(item_boxes)
    return item_dict_list

In [74]:
item_dict_list = []
for page in range(1, 2):
    item_dict_list_sub = get_item_dict_list(search="avengers lunchboxes", page=page)
    item_dict_list.extend(item_dict_list_sub)

In [75]:
df = pd.DataFrame(item_dict_list)
df.head()

,condition,name,price,shipping_price,sponsored
0,Brand New,Marvel's Avengers: Infinity War Lunch Box - O...,13.99,Free Shipping,SPONSORED
1,Brand New,Marvel Kawaii Avengers Girls Boys Soft Insulat...,18.99,Free Shipping,None
2,Pre-Owned,Marvel Super Heroes Metal Lunchbox Thermos Set...,59.50,+$12.60 shipping,None
3,Brand New,Marvel Avengers Captain America Shield Shaped ...,16.33,Free Shipping,SPONSORED
4,Brand New,Avengers Thermal Lunch Box New Kid Child's ma...,13.00,Free Shipping,SPONSORED


In [13]:
soup = get_ebay_soup(search='floor lamps')

In [32]:
soup.find_all("span", class_="s-item__price")[0] # querying the entire webpage

<span class="s-item__price">$37.98</span>

In [27]:
# How do I grab all of the item boxes?
# <li class="s-item   " "data-view"="..." "id"="...">
item_boxes = soup.find_all("div", attrs={"class":"s-item__wrapper"})
len(item_boxes)

61

In [28]:
# Pro Tip: Get everything ready for the first item you investigate


In [49]:
item_box_1 = item_boxes[0]
# get price
price = float(item_box_1.find("span", class_="s-item__price").text.strip("$"))
condition = item_box_1.find("span", class_="SECONDARY_INFO").text
# get the name/sponsored boolean
name_text = item_box_1.find("h3").text.replace("SPONSORED", "SPONSORED$&~")
if 'SPONSORED' in name_text:
    sponsored = name_text.replace("SPONSORED", "SPONSORED ").split("$&~")[0]
    name = name_text.replace("SPONSORED", "SPONSORED ").split("$&~")[1]
else:
    name = name_text
    sponsored = None
    
# shipping price
shipping_price = item_box_1.find("span", class_="s-item__shipping s-item__logisticsCost").text
print(sponsored, name, price, shipping_price, condition)

SPONSORED  Wood Shelf Floor Lamp Linen Shade Light Storage Organizer Living Room Black Home 37.98 Free Shipping Brand New


In [50]:
# Now that we're able to get our information from 1 box, let's generalize and load into df

In [ ]:
make

In [54]:
df = pd.DataFrame(dlist)
df.head()

,condition,name,price,shipping_price,sponsored
0,Brand New,Wood Shelf Floor Lamp Linen Shade Light Storag...,37.98,Free Shipping,SPONSORED
1,Brand New,Aged Brass Pharmacy Floor Lamp Adjustable Swin...,99.99,Free Shipping,None
2,Brand New,Floor Lamps Living Room Lamp Arc LED Modern Ov...,54.02,Free Shipping,None
3,Brand New,Floor Lamps Living Room Lamp Arc LED Modern Ov...,49.99,Free Shipping,None
4,Brand New,Modern LED Floor Lamp Reading Light Dimmable R...,37.03,Free Shipping,None


### What did you learn?
* How strip works with strings
* Using sublime text
* how to maneuver through inpsection elements
* Use try/except
* The nature of web scraping is messy

In [ ]:
|